## <font color='###D4EDDA'>Step 1: Import the necessary Python libraries</font>

In [19]:
import requests
import xlsxwriter
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import os

## <font color='##F2F2F2'> Step 2: Define the URL and send an HTTP GET Request</font>

In [20]:
# URL of the website containing the Excel function information
url = "https://support.microsoft.com/en-us/office/excel-functions-alphabetical-b3944572-255d-4efb-bb96-c6d90033e188#bm6"


# Send an HTTP GET request to the URL
response = requests.get(url)

## <font color='##FFF3CD'> Step 3: Parse HTML Content and Define the output directory path</font>

In [21]:
# Check if the request was successful (status code 200)
if response.status_code == 200:
    pass
else:
    print("Failed to retrieve the webpage.")
    
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
        
# Define the directory where you want to save the Excel file
output_directory = "pase/your/filepath/here"
            
# Define the full file path including the directory and file name
output_file_path = os.path.join(output_directory, "All Excel Formulas.xlsx")

# Create a Pandas Excel writer using XlsxWriter as the engine and specify the output file path
excel_writer = pd.ExcelWriter(output_file_path, engine="xlsxwriter")

## <font color='#FF99CC'> Step 4: Apply Formats to Title and Columns</font>

In [22]:
# Find all tables on the page
tables = soup.find_all("table")

# Define formats for the title and data columns
title_format = excel_writer.book.add_format({
    'bold': True,
    'font_name': 'Calibri',  # Change font name here
    'bg_color': '#3366FF',   # Change background color here
    'align': 'center',  # Center-align the text
    'valign': 'vcenter'  # Center-align vertically
})

## <font color='#FFCC00'> Step 5: Extract and Process Tables</font>

In [23]:

    # Iterate through the tables and save them to Excel
    for index, table in enumerate(tables):
        # Convert the table to a Pandas DataFrame
        html_str = str(table)
        html_io = StringIO(html_str)
        df = pd.read_html(html_io)[0]

        # Initialize empty lists to store modified data
        new_columns = ['Function', 'Type', 'Description']
        new_data = []
        
        for row in df.values:
            function = row[0]
            type_desc = row[1]
            type_info, description = type_desc.split(': ', 1) if ': ' in type_desc else ('', type_desc)
            new_data.append([function, type_info, description])

        # Create a new DataFrame with modified data and columns
        new_df = pd.DataFrame(new_data, columns=new_columns)
        
        # Write the DataFrame to the Excel file
        sheet_name = f"Excel_formula{index}"
        new_df.to_excel(excel_writer, sheet_name=sheet_name, index=False)
        
        # Get the worksheet object
        worksheet = excel_writer.sheets[sheet_name]

        # Calculate the maximum width for each column
        num_columns = len(new_columns)
        max_widths = [0] * num_columns
        
        for _, row in new_df.iterrows():
            for col_num in range(num_columns):
                col_name = new_columns[col_num]
                cell_data = str(row[col_name])
                max_widths[col_num] = max(max_widths[col_num], len(cell_data))

        # Set column widths based on the maximum width for each column
        for col_num in range(num_columns):
            worksheet.set_column(col_num, col_num, max_widths[col_num] + 2)  # Add extra space for padding

        # Write column headers
        for col_num, col_name in enumerate(new_columns):
            worksheet.write(0, col_num, col_name, title_format)


## <font color='##ADD8E6'> Step 6: Save Excel file</font>

In [24]:
    # Save the Excel file
excel_writer.close()
    print("Tables extracted with all excel formulas and save as 'All Excel Formulas.xlsx'.")

Tables extracted with all excel formulas and save as 'All Excel Formulas.xlsx'.
